In [14]:
from datetime import date, timedelta
import holidays
import pandas as pd

In [59]:
def calendar(year,country='ES',subdiv=None):
    """
    returns a dataframe with three columns:
    dates: with format [YYYY-MM-DD], ascending
    weekend: with value 1 if it's Saturday or Sunday, otherwise it returns 0.
    holidays: with value 1 if it's public holidays, otherwise it returns 0.
    """
    calendar=pd.DataFrame()

    def daterange(year):
        calendar = pd.DataFrame()
        calendar['dates']=pd.date_range(f'{year}-01-01', f'{year}-12-31', freq='D')
        return calendar

    calendario = daterange(year)

    def weekends(calendar):
        calendar['weekday']= calendar['dates'].apply(lambda x: x.weekday())
        weekend = {0:0,1:0,2:0,3:0,4:0,5:1,6:1}
        calendar['weekend'] = calendar['weekday'].map(weekend)
        calendar.drop(columns='weekday',inplace=True)
        return calendar

    calendario = weekends(calendario)

    def public_holidays(calendar):
        es_cl_holidays = holidays.country_holidays(country,subdiv)
        calendar['holidays']=calendar['dates'].apply(lambda x: x in es_cl_holidays).map(int)
        return calendar
    calendario = public_holidays(calendario)
    return calendario


In [60]:
calendar(2022,'ES','CL')

,dates,weekend,holidays
0,2022-01-01,1,1
1,2022-01-02,1,0
2,2022-01-03,0,0
3,2022-01-04,0,0
4,2022-01-05,0,0
...,...,...,...
360,2022-12-27,0,0
361,2022-12-28,0,0
362,2022-12-29,0,0
363,2022-12-30,0,0
